# Predspracovanie údajov

In [14]:
# imports
import pandas as pd
from sklearn.model_selection import train_test_split

## 1. Realizácia predspracovania dát

In [15]:
connections = pd.read_csv('data/connections_cleaned.csv')
connections['ts'] = pd.to_datetime(connections['ts'])

processes = pd.read_csv('data/processes_cleaned.csv')
processes['ts'] = pd.to_datetime(processes['ts'])

conn_processes = pd.merge(connections, processes, on=['ts', 'imei', 'mwra'], how='inner')
conn_processes.drop_duplicates(inplace=True)

conn_processes.head(2)

,ts,imei,mwra,c.android.gm,c.android.youtube,c.katana,c.dogalize,c.android.chrome,c.android.vending,c.raider,...,p.dogalize,p.android.defcontainer,p.google,p.android.gms,p.olauncher,p.android.vending,p.browser.provider,p.process.gapps,p.gms.persistent,p.simulator
0,2018-05-05 10:00:00,3590433799317661073,1.0,10.73779,7.60358,10.30550,12.72137,17.14262,63.02014,52.99201,...,49.63998,52.22598,86.79731,88.93302,12.84506,2.63115,90.52801,90.51873,35.23943,93.56427
1,2018-05-05 10:01:00,3590433799317661784,1.0,13.88095,10.68630,8.36266,13.90403,13.28416,54.83081,41.20946,...,50.35593,45.19193,87.84568,25.99896,90.22100,71.68478,89.94562,71.77292,64.49298,42.81274


In [16]:
devices = pd.read_csv('data/devices_cleaned.csv')
profiles = pd.read_csv('data/profiles_cleaned.csv')

### (A)

In [17]:
X = conn_processes.drop(columns=['mwra'])
y = conn_processes['mwra']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f'Train Dataset Shape: ', X_train.shape, y_train.shape)
print(f'Test Dataset Shape: ', X_test.shape, y_test.shape)

Train Dataset Shape:  (11835, 32) (11835,)
Test Dataset Shape:  (2959, 32) (2959,)


### (B)

### (C)

### (D)

## 2. Výber atribútov pre strojové učenie

### (A)

### (B)

### (C)

## 3. Replikovateľnosť predspracovania

### (A)

### (B)

<small>Percentuálny podiel práce členov na projekte: Vlastislav Pokorný: 50%, Leon Radó: 50%</small>